In [1]:
import numpy as np
import random
import syft as sy
import json
from syft.core.frameworks.torch import utils

In [2]:
x = sy.array([1,2,3,4])
me = sy.local_worker
hook = me.hook
me.is_client_worker = False

In [3]:
x2 = sy.FloatTensor([1,2,3,4])

In [4]:
bob = sy.VirtualWorker(hook=hook,id=2)

In [5]:
me.add_worker(bob)

In [6]:
xt = x.send(bob)
xt2 = x2.send(bob)
# xt = x.torch().send(bob)

In [7]:
bob._objects

{3187057183: array([1, 2, 3, 4]),
 95464285132: [_LocalTensor - id:95464285132 owner:2]}

In [13]:
res = xt.cumsum(0)
# res

In [14]:
res.get()

array([ 1,  3,  6, 10])

In [10]:
res = xt2.cumsum(0)

In [11]:
res.id

3844000594

In [9]:
bob._objects

{2224853631: array([1, 2, 3, 4]), 3844000594: array([ 1,  3,  6, 10])}

In [10]:
me._objects

{237577038: array([1, 2, 3, 4]),
 1132434330: array_ptr(None, dtype=object),
 3844000594: array([], dtype=float64),
 7623296799: array([], dtype=float64),
 62769686748: array_ptr(None, dtype=object)}

In [28]:
me.de_register(res)

In [29]:
res.owner

<syft.core.workers.virtual.VirtualWorker id:0>